In [1]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from pandas import DataFrame
import urllib
import time

In [7]:
# 텍스트를 읽은 내용이 저장될 빈 변수
source = None

with open('노래가사.txt', 'r', encoding='utf-8') as f:
    source = f.read()

# 텍스트의 내용을 줄바꿈 단위로 잘라낸다.
lines = source.split('$')
lines

['Whiskey and Morphine\n',
 "\nI'm wanted for addiction\nShe's feening for a thrill\nWe've been screamin' for attention, hmm\nAny little thing to numb this hell\n",
 "\nOh every now and then I like to get me some, to get me some\nOh even though it's just a phase\nOh, now I feel like I've been in a hit and run, this shit ain't fun\nI'm still here and you've gone MIA\n",
 "\nI've got whiskey and morphine\nRushing through me\nStand some, move it quicker, then I'm running\nEven trippin' over absolutely nothing\nThanks to whiskey and morphine\nI'm up and then away, up and then away\nUp and then away, oh-uh-oh\n",
 "\nI'll finish every bottle\n'Til I'm empty on the floor\nThen, do it all again tomorrow, hmm\nAny little thing to numb this hell\n",
 "\nOh, every now and then I like to get me some, to get some\nOh, even though it's just a phase\nNow it's feel like I've been slowing a loaded gun\nThis shit ain't fun\nI'm on the verge on painting with my brains\nHelp me\n",
 "\nWhiskey and morphi

In [8]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(5)
driver.get("https://papago.naver.com/")
time.sleep(3)

In [9]:
result = []

prevTrans = None

currentTrans = None

for item in lines:
    item  = item.strip()
    
    if not item:
        continue
        
     # 한글 입력 영역을 식별한다.
    source = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("#txtSource"))
    
    # 기존 입력 내용을 삭제한다.
    source.clear()
    
    # 텍스트 파일에서 읽은 한 줄을 입력한다.
    source.send_keys(item)
    
    # 입력 내용이 적용되는 동안 잠깐의 대기
    time.sleep(0.3)

    # 번역 버튼 식별하기
    trans = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("#btnTranslate"))
    
    # 번역 버튼 클릭
    trans.click()
    
    # 번역이 되는 동안 1초 대기
    time.sleep(1)
    
    while True:
        try:
            # 번역 결과가 출력될 영역을 식별한다.
            target = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("#txtTarget"))
            
            while True:
                
                currentTrans = target.text
                
                if currentTrans:
                    break
                else:
                    time.sleep(1)
            
            if prevTrans != currentTrans:
                print(item)
                print(currentTrans)
                prevTrans = currentTrans
                print('-' * 30)
                sentance = {'eng' : item, 'kor': currentTrans}
                result.append(sentance)
                break
            else:
                time.sleep(2)
                
        except Exception as e:
            currentTrans = "(해석결과 없음)"
            print(e)
            break

Whiskey and Morphine
위스키 모르핀
------------------------------
I'm wanted for addiction
She's feening for a thrill
We've been screamin' for attention, hmm
Any little thing to numb this hell
중독으로 지명 수배 중이다.
그녀는 스릴을 느끼고 있다.
관심을 끌기 위해 소리를 질렀어요
이 지옥을 무감각하게 할 수 있는 어떤 작은 일이라도
------------------------------
Oh every now and then I like to get me some, to get me some
Oh even though it's just a phase
Oh, now I feel like I've been in a hit and run, this shit ain't fun
I'm still here and you've gone MIA
가끔 나도 좀 사주고 싶고, 또 사주고 싶어
오, 비록 하나의 단계이지만
오, 지금 나는 뺑소니친 것 같아. 이건 재미없어.
난 아직 여기 있고 넌 MIA에 갔어
------------------------------
I've got whiskey and morphine
Rushing through me
Stand some, move it quicker, then I'm running
Even trippin' over absolutely nothing
Thanks to whiskey and morphine
I'm up and then away, up and then away
Up and then away, oh-uh-oh
위스키와 모르핀이 있어요
날 헤치고 달려와
조금만 더 서있으면, 더 빨리 움직이면, 난 도망갈 거야.
심지어 아무것도 아닌 것에 걸려 넘어지기도 한다.
위스키와 모르핀 덕분이다.
난 일어나 있고 또 떠나 있고, 올라가고 또 가고
위로 그리고 멀리, 오-어-어-오
------

In [10]:
with open('result.md', 'w', encoding="utf-8") as f:
    for idx, item in enumerate(result):
        f.write(item['eng'])
        f.write("\n")
        f.write("\n")
        f.write("```\n")
        f.write(item["kor"])
        f.write("\n```\n\n")